In [65]:
import yfinance as yf
import math

In [ ]:
stock = yf.Ticker('TSLA')
stockInfo = stock.info

In [182]:
stock.shares
#stock.info['sharesOutstanding']

,BasicShares
Year,
2019,18471336000
2020,17352119000
2021,16701272000


# EPS

In [67]:
def calc_eps(net_income_to_common, shares_outstanding):
    return net_income_to_common / shares_outstanding

calc_eps(stockInfo['netIncomeToCommon'], stockInfo['sharesOutstanding'])

3.038484681151332

# Volatility
Valid intervals: `[1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]`

In [ ]:
def calc_volatility(stock, period, interval):
    historic = stock.history(period=period, interval=interval)
    values = [x for x in historic['Close'] if not math.isnan(x)]
    print(historic['Close'], len(historic['Close']))

    mean = sum(values) / len(values)
    deviation = [price - mean for price in values]
    variance = sum([dev * dev for dev in deviation]) / len(values)
    std_dev = math.sqrt(variance)

    #Not completely sure if this is the exact formula that is needed/wanted?
    volatility = std_dev * math.sqrt(len(values))
    
    return volatility
    


calc_volatility(stock, '2mo', '1d')


# Estimate future price

In [195]:
gr_values = {
    'Total Assets': 0.1,
    'Total Current Assets': 0.05,
    'Retained Earnings': 0.05,
    'Gross Profit': 0.2,
    'Net Income': 0.2,
    'Total Cash From Operating Activities': 0.15
}

def calc_growth_rate(stock, eps_an, periods, bs, fc, cf):
    result = 0

    for i, (data, weight) in enumerate(gr_values.items()):
        #Take the correct source based on the index since dicts are ordered
        if i < 3:
            source = bs
        elif i < 5:
            source = fc
        else:
            source = cf
            
        data_gr = 0
        for j, period in enumerate(periods):
            #Don't run if we cant +1 the index in the same list
            if j == len(periods) - 1:
                break
            
            #Take present and past values of the thing
            present = source[period][data]
            past = source[periods[j + 1]][data]
            #Calculate growth rate
            data_gr += ((present - past) / past)
            
        #Add the average growth rate * weight to the total
        result += ((data_gr / 2) * weight)
        
    #Add EPS growth rate specifically
    eps_gr = 0
    for i, eps in enumerate(eps_an):
        if i == len(periods) - 1:
            break
                    
        past = eps_an[i + 1]
        eps_gr += ((eps - past) / past)
    result += ((eps_gr / 2) * 0.25)

    return result


#returns stockprice in 1/3 years
def get_future_price(stock, when='3y'):
    #setting the correct financial statement sheets
    #settings the correct keyword arguments for the history period
    #setting the correct list splice length for the periods
    if when == '1y':
        financials = stock.quarterly_financials
        bs = stock.quarterly_balancesheet
        cf = stock.quarterly_cashflow
        kwargs = {'period': '1y'}
        splice = 4
    elif when =='3y':
        financials = stock.financials
        bs = stock.balancesheet
        cf = stock.cashflow
        kwargs = {'start':'2019-01-01', 'end':'2021-12-31'}
        splice = 5
    else:
        print("'when' param should be 1y or 3y")
        return -1

    #Take the historic data / filter the NaN values / splice to the correct amount of periods
    historic = stock.history(interval='1mo', **kwargs)['Close']
    fc = historic[historic.notnull()]
    periods = financials.keys()[:splice]

    #Calculate the EPS for every period in the periods list
    eps_an = [round(financials[period]['Net Income Applicable To Common Shares'] / stock.shares['BasicShares'][period.year if period.year < 2022 else 2021], 2) for period in periods]

    # Calculate future p/e ratio
    #take the relevant closing prices for the periods
    if when == '1y':
        closes = [fc[ts] for ts in fc.keys() if ts.month % 3 == 0]
    elif when =='3y':
        closes = [fc[ts] for ts in fc.keys() if ts.month == 12]
    #in the correct order
    closes.reverse()

    #calculate p/e ratio for every period
    tot_pe = [price / eps_an[i] for i, price in enumerate(closes)]
    #take the average p/e ratio
    pe = round(sum(tot_pe) / len(tot_pe), 2)

    #Calculate future eps (eps * (1 + growth)_rate^periods)
    eps = eps_an[0] * ((1 + calc_growth_rate(stock, eps_an, periods, bs, financials, cf)) ** len(periods))

    #return future p/e ratio * eps
    return pe * eps
    
when = '1y'
print(f"Future {when} price of {stock.info['symbol']}: {get_future_price(stock, when)}")

Future 1y price of AAPL: 207.39551706274975
